<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Projet Data Science</h1></center>
<center><h2>Création de jeux de données - visite</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Nous allons à présent modéliser les visites de nos utilisateurs. En effet, nous ne nous sommes pas encore intéressé 

    
Ainsi dans notre exemple on considérera qu'un visiteur commence une visite à partir du moment où il visite une page. La visite continue tant que le visiteur provoque des événements sur le site et elle n'est interrompue que si le visiteur arrête toute activité pendant 24h. A partir de ce moment il commencera une seconde visite s'il revient sur le site internet. <br><br>
Nous allons essayer de voir si un individu qui commence une visite à un moment donné va finir par acheter un produit le dans les 3 mois qui suivent: si le visiteur fait un achat lors de sa première visite, on ne le considère pas comme un acheteur. Seul compte la visite suivante. <br><br>
On est donc face à un problème de classification. Il va y avoir beaucoup de travail de modélisation des données.

<div class="alert-info"> Cette partie étant un peu complexe en termes d'algorithmes, on peut faciliter la programmation en ne gardant qu'une partie des données pour coder et seulement à la fin utiliser toutes les données une fois que nous sommes sûrs que le code est bon.</div>
</blockquote>


* importer <code>pandas</code>, <code>numpy</code>, et <code>datetime</code>
* charger le jeu de données à partir du fichier <code>events_light.csv</code> (l'index est donné par la première colonne)
* créer la colonne <code>datetime</code> en changeant la colonne <code>timestamp</code> en <code>datetime</code>



In [ ]:
# Insérer le code ici


In [2]:
# Insérer le code ici

import pandas as pd 
import numpy as np
import datetime

df = pd.read_csv('events_light.csv',index_col=0)
df['datetime'] = df['timestamp'].apply(lambda ts: datetime.datetime.fromtimestamp(ts / 1000))

df.head()

/home/paul/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12.117
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14.164
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19.827
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35.914
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17.106


* trier le jeu de données par <code>visitorid</code> puis par <code>timestamp</code>

In [3]:
# Insérer le code ici


In [4]:
# Insérer le code ici

df = df.sort_values(by=['visitorid', 'timestamp'])
df.head()

,timestamp,visitorid,event,itemid,transactionid,datetime
1327067,1442004589439,0,view,285930,NaN,2015-09-11 22:49:49.439
1332475,1442004759591,0,view,357564,NaN,2015-09-11 22:52:39.591
1332601,1442004917175,0,view,67045,NaN,2015-09-11 22:55:17.175
808471,1439487966444,1,view,72028,NaN,2015-08-13 19:46:06.444
722518,1438969904567,2,view,325215,NaN,2015-08-07 19:51:44.567


* créer une colonne <code>timestamp_diff</code> qui contient la différence successive entre deux éléments de la colonne <code>timestamp</code>. Cette colonne sera exprimée en minutes (utiliser la méthode <code>diff</code>)

In [5]:
# Insérer le code ici


In [6]:
# Insérer le code ici

df['timestamp_diff'] = df['timestamp'].diff() / 1000 / 60
df.head()

,timestamp,visitorid,event,itemid,transactionid,datetime,timestamp_diff
1327067,1442004589439,0,view,285930,NaN,2015-09-11 22:49:49.439,NaN
1332475,1442004759591,0,view,357564,NaN,2015-09-11 22:52:39.591,2.835867
1332601,1442004917175,0,view,67045,NaN,2015-09-11 22:55:17.175,2.626400
808471,1439487966444,1,view,72028,NaN,2015-08-13 19:46:06.444,-41949.178850
722518,1438969904567,2,view,325215,NaN,2015-08-07 19:51:44.567,-8634.364617


<blockquote>
Nous devons maintenant trouver un algorithme pour définir les visites. Pour chaque utilisateur, nous allons avoir deux phases:
<ul>
    <li>Lors de la première phase, on parcourt les événements propres à cet utilisateur dans l'ordre anti-chronologique. A chaque fois que nous trouvons une transaction, on considère que cette transaction est déclenche l'ouverture d'une fenêtre de temps lors de laquelle tous les événements que l'on rencontre sont alors comptés comme faisant partie de cette visite. Comme cette visite se termine par une transaction, c'est donc une visite convertissante.</li>
    <li>Lors de la seconde phase, nous parcourons ces événements dans l'ordre chronologique en omettant les événements concernés par les visites convertissantes de la première étape. Lorsque nous rencontrons un élément, nous ouvrons une fenêtre de temps dans laquelle nous conserverons les éléments qui sont à l'intérieur de cette fenêtre.</li>
</ul>
Pour mieux expliquer cette méthode, on peut se référer au schéma suivant: 
<center>
<img src="./visits.gif">
</center>

Cette approche soulève cependant certains problèmes:
<ul>
    <li>si un événement arrive juste avant le début de la fenêtre d'une visite convertissante, on voudrait la compter dans cette visite. Cependant il faut bien définir une limite pour ces visites donc nous ne changerons pas ce point.</li>
    <li>si une seconde transaction a lieu alors que la fenêtre temporelle n'est pas terminée pour une visite convertissante, il faudrait alors compter les événements comme appartenant à une nouvelle visite convertissante.</li>
</ul>
    
Comme toute modélisation, celle-ci n'est pas parfaite mais elle a le mérite d'être simple. 

La fonction qui calcule ces visites est donnée dans la cellule suivante. 
</blockquote>

* inspecter le contenu de la cellule et la lancer

In [7]:
# éxécuter le contenu de la cellule

def create_visits(list_of_events, time_window=2 * 60 * 60 * 1000): 
    '''
    fonction qui renvoie une liste de visites convertissantes et une liste de visites non convertissantes pour un utilisateur
    
    :param list_of_events: liste de dictionnaires représentant les événements d'un seul utilisateur classés par timestamp. 
                            Ces dictionnaires doivent avoir au moins les clefs 'id', 'timestamp' et 'event_type'
    :param time_window: entier qui représente la durée maximale d'une visite en minutes
    :return: une liste de visites convertissantes et une liste de visites non convertissantes pour un utilisateur
    
    '''
    
    # renversons la liste des evenements
    list_of_events_reversed = list_of_events[::-1]    
    
    # on initialise les valeurs 
    # fin de visite
    visit_end = None
    # début de visite
    visit_start = None
    # liste des visites convertissantes
    converting_visits = []
    # liste des visites non convertissantes
    non_converting_visits = []
    # liste des événements restant
    remaining_events = []
    # liste des événements de la visite courante 
    current_visit = []
    
    # parcours des événements en ordre anti-chronogiques pour trouver les visites convertissantes
    for event in list_of_events_reversed: 
        # si l'événement est une transaction, on peut commencer une visite convertissante
        if event['event_type'] == 'transaction':
            
            # si la liste d'événements de la visite courante n'est pas vide on l'ajoute à la liste des visites convertissantes
            if len(current_visit) > 0: 
                converting_visits.append(current_visit)

            # on initialise l'événement avec la transaction
            current_visit = [event]
            
            # on redéfinit la date limite
            visit_end = event['timestamp']

        # si on a affaire à un autre type d'événement 
        else: 
            
            # on vérifie qu'on est bien dans la limite de temps 
            if (visit_end is not None) and (visit_end - event['timestamp']) < time_window:
            
                # si oui on ajoute l'événement à la liste des événements de la visite courante
                current_visit = [event] + current_visit  # on en profite pour remettre les événements dans l'ordre chronologique
                
            else: 
                # si non il faut arrêter la visite convertissante courante
                if len(current_visit) > 0: 
                    converting_visits.append(current_visit)
        
                # réinitialiser les données de la visite courante
                visit_end = None
                current_visit = []
                
                # et ajouter l'événement aux événements restant
                remaining_events = [event] + remaining_events
        
    # dans le cas où il reste une liste convertissante après la boucle
    if len(current_visit) > 0: 
        converting_visits.append(current_visit)
    current_visit = []
        
    # on parcourt les événements restants
    for event in remaining_events: 

        # si l'événement est dans la fenetre
        if (visit_start is not None) and (event['timestamp'] - visit_start < time_window): 
            current_visit.append(event)

        # si l'heure de depart n'est pas définie ou si la fenetre est dépassée
        else: 

            # on ajoute la visite courante à la liste des visites non convertissantes 
            if len(current_visit) > 0: 
                non_converting_visits.append(current_visit)
            # on recommence les visites
            current_visit = [event]
            visit_start = event['timestamp']

    # on ajoute la dernière visite
    if len(current_visit) > 0: 
        non_converting_visits.append(current_visit)
            
    return converting_visits, non_converting_visits
        
        

<blockquote>
    Dans la cellule suivante on définit des données aléatoires pour vérifier que notre fonction marche correctement. 
</blockquote>

* éxécuter la cellule suivante 

In [8]:
# éxécuter cette cellule 

import numpy as np
from  pprint import pprint 

# generons des données aléatoires 
data = {
    'id' : [i for i in range(10000)],
    'timestamp' : sorted(np.random.randint(low=0, high=100000, size=10000)),
    'event_type' : np.random.choice(a=['transaction', 'addtocart', 'view'], p=[.02, .2, .78], size=10000)
}

# créons une liste d'événements
events = [dict(item) for _, item in pd.DataFrame(data).iterrows()]



# faisons le calcul sur toute la liste
converting, non_converting = create_visits(events)
print('nb of transactions: ', sum(data['event_type'] == 'transaction'))
print('nb of converting visits', len(converting))

# faisons le calcul sur les dix premiers éléments
converting, non_converting = create_visits(events[:10], time_window=60)
# on imprime les 10 premiers éléments
pprint(events[:10])
print('converting visits:')
pprint(converting)
print('non converting visits:')
pprint(non_converting)

nb of transactions:  200
nb of converting visits 200
[{'event_type': 'view', 'id': 0, 'timestamp': 20},
 {'event_type': 'addtocart', 'id': 1, 'timestamp': 69},
 {'event_type': 'view', 'id': 2, 'timestamp': 73},
 {'event_type': 'view', 'id': 3, 'timestamp': 87},
 {'event_type': 'view', 'id': 4, 'timestamp': 98},
 {'event_type': 'view', 'id': 5, 'timestamp': 99},
 {'event_type': 'addtocart', 'id': 6, 'timestamp': 126},
 {'event_type': 'view', 'id': 7, 'timestamp': 130},
 {'event_type': 'view', 'id': 8, 'timestamp': 142},
 {'event_type': 'view', 'id': 9, 'timestamp': 188}]
converting visits:
[]
non converting visits:
[[{'event_type': 'view', 'id': 0, 'timestamp': 20},
  {'event_type': 'addtocart', 'id': 1, 'timestamp': 69},
  {'event_type': 'view', 'id': 2, 'timestamp': 73}],
 [{'event_type': 'view', 'id': 3, 'timestamp': 87},
  {'event_type': 'view', 'id': 4, 'timestamp': 98},
  {'event_type': 'view', 'id': 5, 'timestamp': 99},
  {'event_type': 'addtocart', 'id': 6, 'timestamp': 126},
  

<blockquote>
    Bien sûr ce test n'est pas exhaustif mais il permet de voir certains problèmes qui pourraient être soulevés par cette fonction.<br>
    Il faut maintenant créer les données sur le vrai jeu de données. <br>
    <br>
    Dans un premier temps, recréons la colonne <code>event_id</code> à partir de l'index. 
    Puis, pour chaque utilisateur, nous devons construire une liste qui comprend
</blockquote>

* exécuter la cellule suivante

In [9]:
# Exécuter cette cellule 

# on recrée la colonne eventid
df = df.reset_index()
df = df[['visitorid', 'event', 'timestamp', 'index']]
df.columns = ['visitorid', 'event_type', 'timestamp', 'eventid']

# on crée un colonne data qui contient les données relatives à un événement
df['data'] = [dict(item) for _, item in df.iterrows()]

# on aggrége ces données par visiteurs pour obtenir la liste des événements par visiteur
df_visits = df.groupby(['visitorid']).agg({'data': list})

# on crée une colonne `visits` avec la liste des visites convertissantes et non convertissantes
df_visits['visits'] = df_visits['data'].apply(lambda list_of_events: create_visits(list_of_events))

# on sépare les visites non convertissantes des visites convertissantes
df_visits['converting_visits'] = df_visits['visits'].apply(lambda visits: visits[0])
df_visits['non_converting_visits'] = df_visits['visits'].apply(lambda visits: visits[1])

# comme le calcul est assez long on va sauvegarder nos résultats
df_visits.to_csv('visits.csv')

<blockquote>
Ces calculs étant assez longs, on a fait une sauvegarde qui nous permet de reprendre à partir d'ici:
</blockquote>

* importer le jeu de données <code>visit.csv</code> et le nommer <code>df_visits</code>
* n'importer que les colonnes <code>visitorid</code>, <code>non_converting_visits</code> et <code>converting_visits</code>
* appliquer la fonction <code>literal_eval</code> de la librairie <code>ast</code> au deux dernières colonnes du DataFrame 

In [10]:
# Insérer votre code ici


In [11]:
# Insérer votre code ici

import pandas as pd 
import numpy as np
df_visits = pd.read_csv('visits.csv', usecols=['visitorid', 'non_converting_visits', 'converting_visits'])

import ast 
df_visits['non_converting_visits'] = df_visits['non_converting_visits'].apply(ast.literal_eval)
df_visits['converting_visits'] = df_visits['converting_visits'].apply(lambda s: ast.literal_eval(s.strip()))

df_visits.head()

,visitorid,converting_visits,non_converting_visits
0,0,[],"[[{'visitorid': 0, 'event_type': 'view', 'time..."
1,1,[],"[[{'visitorid': 1, 'event_type': 'view', 'time..."
2,2,[],"[[{'visitorid': 2, 'event_type': 'view', 'time..."
3,3,[],"[[{'visitorid': 3, 'event_type': 'view', 'time..."
4,4,[],"[[{'visitorid': 4, 'event_type': 'view', 'time..."


<blockquote>
    On veut à présent un jeu de donnée à la maille visite:
</blockquote>

* créer une liste de visites nommées <code>visits</code> qui contiendra les éléments des colonnes <code>converting_visits</code> et <code>non_converting_visits</code>
* transformer cette liste en un <code>pd.DataFrame</code> nommé <code>df_visits_clean</code>
* ajouter une colonne <code>converting</code> qui vaut 1 quand la visite est convertissante, 0 sinon
* ajouter une colonne <code>visitorid</code> qui contient l'id du visiteur qui fait la visite
* ajouter une colonne <code>visitid</code> qui contient un id pour chaque visite

In [12]:
# Insérer votre code ici


In [13]:
# Insérer votre code ici

non_converting_visits = [visit for visitor_visits in df_visits['non_converting_visits'] for visit in visitor_visits]
converting_visits = [visit for visitor_visits in df_visits['converting_visits'] for visit in visitor_visits]

nb_converting_visits = len(converting_visits)

df_visits_clean = pd.DataFrame({'visits': converting_visits + non_converting_visits})
df_visits_clean['converting'] = 0
df_visits_clean.loc[:nb_converting_visits, 'converting'] = 1

df_visits_clean['visitorid'] = df_visits_clean['visits'].apply(lambda visit: visit[0]['visitorid'])

df_visits_clean['visitid'] = df_visits_clean.index

df_visits_clean.head()

,visits,converting,visitorid,visitid
0,"[{'visitorid': 172, 'event_type': 'transaction...",1,172,0
1,"[{'visitorid': 172, 'event_type': 'view', 'tim...",1,172,1
2,"[{'visitorid': 186, 'event_type': 'addtocart',...",1,186,2
3,"[{'visitorid': 264, 'event_type': 'transaction...",1,264,3
4,"[{'visitorid': 264, 'event_type': 'view', 'tim...",1,264,4


<blockquote>
    Maintenant nous allons juste nettoyer nos données afin de les alléger pour plus tard. <br>
    Nous pourrons toujours retrouver ces données grâce à l'id des événements. 
</blockquote>

* exécuter la cellule suivante

In [14]:
# Exécuter le contenu de la cellule

df_visits_clean['visits'] = df_visits_clean['visits'].apply(lambda visit: list(map(lambda evenement: evenement.get('eventid'), visit)))
df_visits_clean.head()

<blockquote>
<p>Sauvegardons les données pour faire une exploration plus en profondeur.     
</blockquote>

* sauvegarder le jeu de données sous le nom <code>visits_light.csv</code>

</blockquote>

In [12]:
# Insérer votre code ici


In [12]:
# Insérer votre code ici

df_visits_clean.to_csv('visits_light.csv', index=False)